# 1.1 Prompt Engineering

System prompts are the easiest way to modify an agent's performance and persona.  

• Role Setting: Using a system prompt can change the agent's behavior (e.g., making it act as a "science fiction writer" to answer questions it normally couldn't).  
• Few-Shot Prompting: You can provide input-output pair examples within the prompt to force the agent to follow a specific style or length.  
• Structural Constraints: Explicitly defining the structure (e.g., asking for four specific topics) prevents the LLM from hitting "self-determined word counts" and keeps answers high-quality.   

Let's see the different techniques in the following examples.

## 1.1.0 Getting Started

Load the required environment variables:

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## 1.1.1 Basic Prompting

In [ ]:
import os
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
)
agent = create_agent(model=model)

question = HumanMessage(content="What's the capital of the moon?")

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

In [ ]:
system_prompt = "You are a science fiction writer, create a capital city at the users request."

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

## 1.1.2 Few-shot examples

In [ ]:
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

User: What is the capital of mars?
Scifi Writer: Marsialis

User: What is the capital of Venus?
Scifi Writer: Venusovia

"""

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

## 1.1.3 Structured prompts

In [ ]:
system_prompt = """

You are a science fiction writer, create a space capital city at the users request.

Please keep to the below structure.

Name: The name of the capital city

Location: Where it is based

Vibe: 2-3 words to describe its vibe

Economy: Main industries

"""

scifi_agent = create_agent(
    model=model,
    system_prompt=system_prompt
)

response = scifi_agent.invoke(
    {"messages": [question]}
)

print(response['messages'][1].content)

## 1.1.4 Structured output

For more advanced use cases, you can move beyond text blocks by providing an output schema. This forces the model to fill out specific fields (like a form).  
Structured output is vital if you intend to use the agent's response as data for functions down the line, allowing you to pull specific values programmatically.

In [ ]:
from langchain.agents import create_agent
from langchain.messages import HumanMessage
from pydantic import BaseModel

class CapitalInfo(BaseModel):
    name: str
    location: str
    vibe: str
    economy: str

agent = create_agent(
    model=model,
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
    response_format=CapitalInfo
)

question = HumanMessage(content="What is the capital of The Moon?")

response = agent.invoke(
    {"messages": [question]}
)

response["structured_response"]

In [ ]:
response["structured_response"].name

In [ ]:
capital_info = response["structured_response"]

capital_name = capital_info.name
capital_location = capital_info.location

print(f"{capital_name} is a city located at {capital_location}")